In [7]:
import numpy as np
from ortools.sat.python import cp_model


NUM_MEALS = 5
MEAL_TYPES = ['breakfast', 'dinner', 'lunch', 'snack']
REQUIRED_MEALS = [1, 1, 1, 2]
TARGET_CALORIES = 2_000
TOLERANCE = 50

In [8]:
def make_random_meal_options():
    options = []

    for _ in range(200):
        calories = np.random.randint(100, 1_000)
        num_types = np.random.randint(1, len(MEAL_TYPES))
        types = np.random.choice(MEAL_TYPES, num_types, replace=False)

        options.append(dict(calories=calories, types=types,name='belveta'))

    # List of dictionaries with keys 'calories' and 'types'
    return options

meal_options = make_random_meal_options()

In [9]:


calories = np.array([option['calories'] for option in meal_options])

# Element (i, j) == True iff meal i has meal type j, and False otherwise.
meal_types = np.empty((len(meal_options), len(MEAL_TYPES)), dtype=bool)

for i, option in enumerate(meal_options):
    for j, meal_type in enumerate(MEAL_TYPES):
        meal_types[i, j] = meal_type in option['types']

In [10]:

model = cp_model.CpModel()

# Decision variables, one for each meal and meal type: meal[i, j] is 1 iff
# meal i is assigned to meal type j, and 0 otherwise.
meal_vars = np.empty((len(meal_options), len(MEAL_TYPES)), dtype=object)

for i in range(len(meal_options)):
    for j in range(len(MEAL_TYPES)):
        meal_vars[i, j] = model.NewBoolVar(f"meal[{i}, {j}]")

# We want the overall caloric value of the meal plan to be within bounds.
lb, ub = [TARGET_CALORIES - TOLERANCE, TARGET_CALORIES + TOLERANCE]
model.AddLinearConstraint(calories @ meal_vars.sum(axis=1), lb, ub)

for j, meal_type in enumerate(MEAL_TYPES):
    # Need the required amount of meals of each type.
    model.Add(meal_types[:, j] @ meal_vars[:, j] == REQUIRED_MEALS[j])

for i in range(len(meal_options)):
    # Each meal can only be selected once across all meal types.
    model.Add(meal_vars[i, :].sum() <= 1)

# Need NUM_MEALS meals in the meal plan
model.Add(meal_vars.sum() == NUM_MEALS)

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"Solving took {solver.WallTime():.2f} seconds")

    for i in range(len(meal_options)):
        for j in range(len(MEAL_TYPES)):
            if solver.Value(meal_vars[i, j]) > 0:
                option = meal_options[i]
                cal = option['calories']
                mt = MEAL_TYPES[j]

                print(f"Selected meal {option['name']} {i} with {cal} calories for {mt}.")
else:
    print("No solution found.")

Solving took 0.01 seconds
Selected meal belveta 92 with 339 calories for snack.
Selected meal belveta 97 with 257 calories for breakfast.
Selected meal belveta 132 with 119 calories for snack.
Selected meal belveta 143 with 857 calories for lunch.
Selected meal belveta 161 with 456 calories for dinner.
